<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/R/01RAD_Ex06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 06


Today's exercise
 * Multivariable Regression models
 * Introduction to Residual Analysis and Posthoc Annalysis

In [ ]:
# get requirements for this exercise
list_of_packages <- c("tidyverse", "lattice", "pwr", "MASS",
                      "GGally","scatterplot3d","gridExtra",
                      "nortest","lmtest","colorspace", "car","pracma") # "car","colorspace","RColorBrewer","pracma"
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
######################
# check our settings
# getwd()
# print(.libPaths())
# print(sessionInfo())
print(version)

tibble::tibble(
  Package = names(installed.packages()[,3]),
  Version = unname(installed.packages()[,3])
)


##  Multiple regression intro


In [ ]:
# We use again trees data to present Multivariable linear regression 
data("trees")
head(trees)

In [ ]:
str(trees)
summary(trees)
dim(trees)

In [ ]:
# Lets boost our data.frame and add tree species
# We generate them random:
set.seed(20)

# Based on binomial ditribution
#trees$Species <- factor(rbinom(nrow(trees),3,0.5)+1,
#                         labels=(c("oak","spruce","maple","pine")))

# Based on multinomial distribution with prob=c(0.3,0.3,0.2,0.2)
trees$Species <- factor(max.col(t(rmultinom(nrow(trees),1,prob = c(0.3,0.3,0.2,0.2))), 'first'),
                        labels=(c("oak","spruce","maple","pine")))
summary(trees)
# check what we generate with given seed: summary(lm(Volume ~ (.), data = trees))



faktorová premenná- triedy


In [ ]:
# Delete and reset plotting settings
dev.off()
par(mfrow = c(1, 1))

In [ ]:
# Set colors and graphics points from library(colorspace)
PCH  <- c(15, 16, 17, 18)
COL  <- heat_hcl(4)                                            
BGC  <- diverge_hcl(4)
#COL <- brewer.pal(nlevels(as.factor(trees$Species)), "Set2")  # from library(RColorBrewer)
names(PCH) <- names(COL) <- names(BGC) <- levels(trees[, "Species"])
Xlab <- "Height [ft]"
Ylab <- "Volume [cubic ft]"
Xlim <- range(trees[, "Height"])
Ylim <- range(trees[, "Volume"])

levels(trees$Species)
summary(trees$Species)

In [ ]:
### SCATTERPLOTS

## Basic scatterplot
pairs(trees,  main="Basic Scatterplot Matrix")                      # all variables in data tree
pairs(~(.), data=trees,  main="Basic Scatterplot Matrix")           # all variables in data tree 
pairs(~Girth+Volume,data=trees,  main="Basic Scatterplot Matrix")   # Girth and Volume variables from data tree


je tam krásna lineárna závislosť

In [ ]:
## Scatterplot Matrices  from lattice package
splom(trees)  
splom(trees, groups=trees$Species )  
splom(trees[c("Girth","Height","Volume")], groups=trees$Species )  


In [ ]:
## Scatterplot Matrices from the car package
scatterplotMatrix(~(.)|Species, data=trees,   main="Three Cylinder Options")

In [ ]:
# Tune colors and remove regression lines
scatterplotMatrix(~(.)|Species, data=trees , 
                  reg.line="" , smoother="", col=COL , smoother.args=list(col="grey") ,
                  cex=1.5 , pch=PCH , main="Scatter plot with Three Cylinder Options")

prekladá 4 LM naraz 


In [ ]:
### Plot of Dependence of Trees Volume on Trees Height by Specises
plot(Volume ~ Height, data = trees, 
     pch = PCH[Species], col = COL[Species], bg = BGC[Species],
     xlab = Xlab, ylab = Ylab, main="Dependence of Trees Volume on Trees Height")
legend(63,75, legend = levels(trees[, "Species"]), pch = PCH, col = COL, pt.bg = BGC, y.intersp = 1.2)

In [ ]:
### A ggplot2 generalized pairs plot from GGally (Extension to 'ggplot2')
ggpairs(trees)

In [ ]:
### Plots by categorical variable

# xyplot (Species as factor)
xyplot(Volume ~ Girth  | Species , data=trees , pch=20 , cex=2 , col="blue" )
xyplot(Volume ~ Height | Species , data=trees , pch=20 , cex=2 , col="blue" )

=> nechcem to odhadovať naraz, ale roztrhnem podľa príslušnosti a urobím zvlášť 4 jednoduché LM -> ale to može niesť k strate inf, horšie koef.


In [ ]:
# If species numeric
is.factor(trees$Species)
trees$Species <- as.numeric(trees$Species)
is.factor(trees$Species)
# Different 
xyplot(Volume ~ Girth | Species , data=trees , pch=20 , cex=2 , col="red" )
# back to factor variable :)
trees$Species <- factor(trees$Species, labels=(c("oak","spruce","maple","pine")))
summary(trees)

tie čiary v hlavičách - . labels kde to spadá, do akej kategórie -> podľa levels(factor prem) jakoby na pravítku zárezy.

In [ ]:
#Make the margin around each graph a bit smaller
op <- par(mfrow=c(1,2),oma = c(0, 0, 2, 0))
    # oma is a vector of the form c(bottom, left, top, right)
    # giving the size of the outer margins in lines of text.

#Classical histogram and density
hist(trees$Volume,  main="" , breaks=10 , col="gray" , xlab="Volume" , ylab="Number of trees in each bin")
hist(trees$Volume,  freq=F,  breaks=10, main="", xlab="Volume",xlim=c(-10, 100),ylim=c(0, 0.05))
lines(density(trees$Volume, kernel = "epanechnikov"), col="red", lwd=2)

#Add only ONE title :
mtext("Histogram and Density plot of Volume", outer = TRUE, cex = 1.4, font=4, col=rgb(0.1,0.3,0.5,0.5) )
## At end of plotting, reset to previous settings:
par(op)


jadrový odhad hustotu -> dens_estim : do každej x odhad funkcnej hodnoty, dens_estim\$x ->tie body v ktorych to robim

In [ ]:
# are under the curve - polyarea integration by function from library(pracma)
dens_estim <- density(trees$Volume, kernel = "gauss", from = 0, bw = 4) #kernel="epanechnikov"
trapz(dens_estim$x,dens_estim$y)

In [ ]:
summary(trees$Volume)

In [ ]:
#simulation from density estimation

N = 10000
x_new <- rnorm(N, sample(trees$Volume, size = N, replace = TRUE), dens_estim$bw)
summary(x_new)
plot(dens_estim)
lines(density(x_new), col = "blue")



In [ ]:
# How to split screen (again?)
# Divide the screen in 2 line and 1 column only
new_screen_step1 <- split.screen(c(2, 1))
# Add one graph on the screen number 1 which is on top :
screen(new_screen_step1[1])
plot( Volume~Girth , data=trees, pch=20 , xlab="Girth",ylab="Volume", cex=1 , col="black" )
# I divide the second screen in 2 columns :
new_screen_step1=split.screen(c(1, 2), screen = new_screen_step1[2])
screen(new_screen_step1[1])
hist(trees$Girth , breaks = 8, border=F , col="blue" , main="" , xlab="Distribution of Girth")
screen(new_screen_step1[2])
hist(trees$Volume, breaks = 8, border=F , col="red" , main="" ,  xlab="Distribution of Volume")


In [ ]:
table(trees$Height)
trees$Height_fac<-cut(trees$Height, seq(60,90,10), right=FALSE) # cut creates cathegorical varaible
is.factor(trees$Height_fac)
summary(trees)


In [ ]:
# ggplot version of scatterplot, where Color and shape depend on factor variable
# multiplot with ggplot is different
# more possibilities exist, here is only one
par(mfrow = c(1, 1))
plot1 <- ggplot(trees, aes(x=Girth, y=Volume, color=Species, shape=Species)) + 
    geom_point(size=5, alpha=0.8)  +   theme_bw() 
plot2 <- ggplot(trees, aes(x=Girth, y=Volume, color=Height_fac, size=Height)) + 
    geom_point(alpha=0.6) +   theme_bw() 
grid.arrange(plot1, plot2, ncol=2)

# Reset plot settings
#dev.new()
dev.off()



### Lets start with regression

In [ ]:
trees_lm1.0  <- lm(Volume ~  Species+Girth, data = trees)  # use Species and Girth only (aditive model)
summary(trees_lm1.0)


lm( odozva ~ nezávislé premenné )
  (objem závisí na druhy a obnodu)

znamienka v pravej strane lm()

  \+ pridáva premennu
  
  \: interakcia -> súčin premenných
  \* to mi to vezme aj samostatne aj v interakcií (kombinácia \+ a \:)

  (.) vezme všetky premenné ale len s plusom \+

In [ ]:
#### Model selection by adding variables #########
trees_lm1.0  <- lm(Volume ~ (.), data = trees)  # use all what I have in the dataframe
summary(trees_lm1.0)

oak je ukrytý v intercepte <- výpis ukazuje len 3 a nie 4 factor druhy

In [ ]:
trees_lm1.0  <- lm(Volume ~ (.)^2, data = trees)  # use all what I have in the dataframe 
                                                  # with second order interactions
summary(trees_lm1.0)

keď násobíme numer:faktor -> prechod - ako sa to líši β1 od β 2

NA-> nemám to v dátach, nemožno to odhadnúť

**Question:** Why we have a row with NA in the previous summary table?

Uff - Not this way, lets start from the simplest model. Only Girth and Height as a independent variable

In [ ]:
trees_lm1.0  <- lm(Volume ~ Girth + Height, data = trees)  # use all what I have in the dataframe
# nearly all information is in summary
summary(trees_lm1.0)


In [ ]:
# Construct X,Y, beta_hat - if you want to analyze it by hand
n = nrow(trees)
p = length(coefficients(trees_lm1.0))
beta_hat = as.matrix(coefficients(trees_lm1.0))
X        = as.matrix(cbind(rep(1,times = n), trees[,c(1,2)]))
Y        = as.matrix(trees[,3])
residuals_lm1.0 = Y - X%*%beta_hat
sum(((residuals_lm1.0 - residuals(trees_lm1.0))<0.00001))
# and so on 


In [ ]:
summary(trees)
dim(trees)
trees_new <- data.frame(Girth = (seq(5,25,0.1)), Height = seq(60,90,length=length(seq(5,25,0.1))))
#trees_new <- data.frame(Girth = (rep(10,times=length(seq(5,25,0.1)))), Height = seq(60,90,length=length(seq(5,25,0.1))))


In [ ]:
conf_new  <- predict(trees_lm1.0, newdata = trees_new, interval = "confidence")
pred_new  <- predict(trees_lm1.0, newdata = trees_new, interval = "prediction")
height_lim = seq(60,90,length=length(seq(5,25,0.1)))


In [ ]:
op <- par(mfrow=c(1,2),oma = c(0, 0, 2, 0))

plot(Volume ~ Girth, data = trees, xlim = c(5,25), ylim = c(5,100),pch=20, col = "black", xaxs="i",yaxs="i",
     main="Trees data: Girth and Volume dependence",xlab="Girth", ylab="Volume")
lines(trees_new[,1], pred_new[,1], col='black')
lines(trees_new[,1], pred_new[,2], col='red')
lines(trees_new[,1], pred_new[,3], col='red')
lines(trees_new[,1], conf_new[,2], col='blue')
lines(trees_new[,1], conf_new[,3], col='blue')
legend("topleft",legend=c("observed","fit","Confidence int","Prediction int"),
       pch=c(20,NA,NA,NA),lty = c(NA,1,1,1),col = c("black","black","blue","red"))

plot(Volume ~ Height, data = trees, xlim = c(60,90), ylim = c(5,100),pch=20, col = "black", xaxs="i",yaxs="i",
     main="Trees data: Height and Volume dependence",xlab="Height", ylab="Volume")
lines(height_lim, pred_new[,1], col='black')
lines(height_lim, pred_new[,2], col='red')
lines(height_lim, pred_new[,3], col='red')
lines(height_lim, conf_new[,2], col='blue')
lines(height_lim, conf_new[,3], col='blue')
legend("topleft",legend=c("observed","fit","Confidence int","Prediction int"),
       pch=c(20,NA,NA,NA),lty = c(NA,1,1,1),col = c("black","black","blue","red"))

par(op)



In [ ]:
#install.packages("modelr")
library(modelr)

In [ ]:
trees %>% 
  ggplot(aes(Girth, Volume)) + 
  geom_line() + 
  ggtitle("Full data = ")

mod_g <- lm(Volume ~ Girth, data = trees)
trees %>% 
  add_predictions(mod_g) %>% #pridavame do dat predikcie
  ggplot(aes(Girth, pred)) + 
  geom_line() + 
  ggtitle("Linear trend + ")

trees %>% 
  add_residuals(mod_g) %>%  #pridat rezidua
  ggplot(aes(Girth, resid)) + 
  geom_hline(yintercept = 0, colour = "white", size = 3) + 
  geom_line() + 
  ggtitle("Remaining pattern")

In [ ]:
# Another design of scatterplot3d
s3d <-with(trees,scatterplot3d(Girth, Height, Volume, pch=16,
                               highlight.3d=TRUE, type="h",
                               main="3D Scatter Plot with Vertical Lines and Regression Planes",
                               angle=135,scale.y=1, xlab="Girth",ylab="Height",zlab="Volume",
                               cex.lab=1.5, cex.axis=1.5, cex.main=1.1, cex.sub=1.5))
s3d$plane3d(trees_lm1.0)



In [ ]:
# Comperable statistics of the model 
summary(trees_lm1.0)$r.squared
summary(trees_lm1.0)$adj.r.squared
summary(trees_lm1.0)$fstatistic



In [ ]:
# polynomila Regression without scaled varialbes 
trees_lm2.0 <- lm(Volume ~ (Height) + I((Girth)^2), data = trees)
summary(trees_lm2.0)


In [ ]:
# polynomial Regression with scaled varialbes
trees_lm2.1 <- lm(Volume ~ scale(Height) + I(scale(Girth)^2), data = trees)
summary(trees_lm2.1)
plot(Volume ~ scale(Girth), data = trees)
lines(fitted(trees_lm2.1) ~ scale(Girth), data = trees)

conf_new  <- predict(trees_lm2.1, newdata = trees_new, interval = "confidence")

##############
# Scaling and trasformations will be covered later 
############

In [ ]:

# Qualitative Explanatory Variables - Species

trees$Tall <- cut(trees$Height, breaks = c(-Inf, 76, Inf), labels = c("no", "yes"))
summary(trees)
treesTall  <- split(trees, trees$Tall)
trees_lm_Tall <- lm(Volume ~ Girth + Tall, data = trees)
summary(trees_lm_Tall)
treesTall[["yes"]]$Fit <- predict(trees_lm_Tall, treesTall[["yes"]])
treesTall[["no"]]$Fit  <- predict(trees_lm_Tall, treesTall[["no"]])
plot(Volume ~ Girth, data = trees, type = "n")
points(Volume ~ Girth, data = treesTall[["yes"]], pch = 1)
points(Volume ~ Girth, data = treesTall[["no"]], pch = 2)
lines(Fit ~ Girth, data = treesTall[["yes"]])
lines(Fit ~ Girth, data = treesTall[["no"]])



In [ ]:
# two different scales
trees_lm_Tall1 <- lm(Volume ~ Girth, data = trees[trees$Tall=="yes",])
trees_lm_Tall2 <- lm(Volume ~ Girth, data = trees[trees$Tall=="no",])
treesTall[["yes"]]$Fit1 <- predict(trees_lm_Tall1, treesTall[["yes"]])
treesTall[["no"]]$Fit2  <- predict(trees_lm_Tall2, treesTall[["no"]])
plot(Volume ~ Girth, data = trees, type = "n")
points(Volume ~ Girth, data = treesTall[["yes"]], pch = 1)
points(Volume ~ Girth, data = treesTall[["no"]], pch = 2)
lines(Fit1 ~ Girth, data = treesTall[["yes"]])
lines(Fit2 ~ Girth, data = treesTall[["no"]])

trees_lm_Tall3 <- lm(Volume ~ Girth*Tall, data = trees)
summary(trees_lm_Tall3)
summary(trees_lm_Tall2)
summary(trees_lm_Tall1)


In [ ]:
trees_lm_Tall_all <-lm(Volume ~ Girth * Tall, data=trees) #* lebo chceme aj interakcie
summary(trees_lm_Tall_all)